# Homework 12

In [70]:
import pandas as pd

## Excercise 1

In [71]:
visit_log = pd.read_csv('src/visit_log.csv', sep=';')

In [72]:
def source_type(row):
    """Defines source type from traffic_source"""

    if row['traffic_source'] in ['yandex', 'google']:
        return 'organic'

    if row['traffic_source'] in ['paid', 'email']:
        if row['region'] == 'Russia':
            return 'ad'
        else:
            return 'other'

    return row['traffic_source']

visit_log['source_type'] = visit_log.apply(source_type, axis=1)

## Excercise 2

In [73]:
urls = pd.read_csv('src/URLs.txt')

In [74]:
news_urls = urls[urls.url.str.contains(r'/\d{8}-')]
news_urls.head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


## Excercise 3

In [75]:
ratings = pd.read_csv('src/ml-latest-small/ratings.csv')

In [76]:
def average_lifetime(form='pd'):
    lifetime = ratings.groupby('userId').\
        agg({'rating': 'count', 'timestamp': lambda timestamp: max(timestamp) - min(timestamp)}).\
        query('rating > 100')

    lifetime['lifetime'] = pd.to_timedelta(lifetime['timestamp'], unit='s')

    if form == 'pd':
        return lifetime.lifetime.mean()
    if form == 'py':
        return lifetime.lifetime.mean().to_pytimedelta()
    if form == 'str':
        return str(lifetime.lifetime.mean().to_pytimedelta())

average_lifetime()

Timedelta('463 days 21:28:27.449612384')

## Excercise 4

In [83]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [106]:
rzd.head()
auto.head()
air.head()
client_base.head(10)

joined = pd.merge(rzd, auto, how='outer', on='client_id')
joined = pd.merge(joined, air, how='outer', on='client_id')
joined.fillna(value=0, inplace=True)
joined.head(10)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [111]:
joined_adress = joined.merge(client_base, on='client_id')
joined_adress['total_revenue'] = joined_adress.apply(lambda row: sum([value for value in row.values if type(value) == float]), axis=1)
joined_adress[['client_id', 'address', 'total_revenue']]

,client_id,address,total_revenue
0,111,Комсомольская 4,1093.0
1,112,Энтузиастов 8а,2810.0
2,113,Левобережная 1а,67766.0
3,114,Мира 14,5857.0
4,115,ЗЖБИиДК 1,1974.0
5,116,Строителей 18,4838.0
6,117,Панфиловская 33,111.0
7,118,Мастеркова 4,173.0
